In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from pylab import rcParams

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

# keras imports - comment them out or do `pip install keras`
# from keras.preprocessing.text import one_hot
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense, Flatten
# from keras.layers.embeddings import Embedding

# gensim for pretrained word2vec model
import gensim

rcParams['figure.figsize'] = 10, 6
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
df = pd.read_csv("data/tidy_anime.csv")
df.shape

(77911, 28)

In [17]:
desired_cols = ['animeID', 'title_english', 'type', 'source', 'producers', 'genre', 'studio',
               'episodes', 'premiered', 'rating', 'score', 'scored_by', 'rank', 'popularity',
               'members', 'favorites', 'synopsis']
truncated_df = df[desired_cols]

,animeID,title_english,type,source,producers,genre,studio,episodes,premiered,rating,score,scored_by,rank,popularity,members,favorites,synopsis
0,1,Cowboy Bebop,TV,Original,Bandai Visual,Action,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
1,1,Cowboy Bebop,TV,Original,Bandai Visual,Adventure,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
2,1,Cowboy Bebop,TV,Original,Bandai Visual,Comedy,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
3,1,Cowboy Bebop,TV,Original,Bandai Visual,Drama,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
4,1,Cowboy Bebop,TV,Original,Bandai Visual,Sci-Fi,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
5,1,Cowboy Bebop,TV,Original,Bandai Visual,Space,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
6,5,Cowboy Bebop: The Movie,Movie,Original,Sunrise,Action,Bones,1.0,NaN,R - 17+ (violence & profanity),8.41,120243,164,449,197791,776,"Another day, another bounty—such is the life o..."
7,5,Cowboy Bebop: The Movie,Movie,Original,Sunrise,Drama,Bones,1.0,NaN,R - 17+ (violence & profanity),8.41,120243,164,449,197791,776,"Another day, another bounty—such is the life o..."
8,5,Cowboy Bebop: The Movie,Movie,Original,Sunrise,Mystery,Bones,1.0,NaN,R - 17+ (violence & profanity),8.41,120243,164,449,197791,776,"Another day, another bounty—such is the life o..."
9,5,Cowboy Bebop: The Movie,Movie,Original,Sunrise,Sci-Fi,Bones,1.0,NaN,R - 17+ (violence & profanity),8.41,120243,164,449,197791,776,"Another day, another bounty—such is the life o..."


In [18]:
# filter out bad titles. Only want titles that have an english name

orig_len = len(truncated_df)
filtered_df = truncated_df[truncated_df['title_english'].notnull()]
new_len = len(filtered_df)
print ("removed {} bad anime after filtering for english titled anime only".format(orig_len - new_len))

# drop NaN rows
filtered_df.dropna(inplace=True)
print ("removed {} bad anime after dropping NaN rows".format(new_len - len(filtered_df)))


removed 30430 bad anime after filtering for english titled anime only
removed 17795 bad anime after dropping NaN rows


c:\users\sidac\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,animeID,title_english,type,source,producers,genre,studio,episodes,premiered,rating,score,scored_by,rank,popularity,members,favorites,synopsis
0,1,Cowboy Bebop,TV,Original,Bandai Visual,Action,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
1,1,Cowboy Bebop,TV,Original,Bandai Visual,Adventure,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
2,1,Cowboy Bebop,TV,Original,Bandai Visual,Comedy,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
3,1,Cowboy Bebop,TV,Original,Bandai Visual,Drama,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
4,1,Cowboy Bebop,TV,Original,Bandai Visual,Sci-Fi,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
5,1,Cowboy Bebop,TV,Original,Bandai Visual,Space,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
16,6,Trigun,TV,Manga,Victor Entertainment,Action,Madhouse,26.0,Spring 1998,PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432,"Vash the Stampede is the man with a $$60,000,0..."
17,6,Trigun,TV,Manga,Victor Entertainment,Sci-Fi,Madhouse,26.0,Spring 1998,PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432,"Vash the Stampede is the man with a $$60,000,0..."
18,6,Trigun,TV,Manga,Victor Entertainment,Adventure,Madhouse,26.0,Spring 1998,PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432,"Vash the Stampede is the man with a $$60,000,0..."
19,6,Trigun,TV,Manga,Victor Entertainment,Comedy,Madhouse,26.0,Spring 1998,PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432,"Vash the Stampede is the man with a $$60,000,0..."


In [5]:
# currently the anime is duplicated, one row per genre per studio. We need to flatten all to one row
# also need to do this for type, source, producers, rating

all_ids = set(filtered_df['animeID'].unique()) # 1.8K anime IDs
print ("{} unique anime".format(len(all_ids)))

id_genre_mapping = {}
for each_id in all_ids:
    genre_list = list(filtered_df[truncated_df['animeID'] == each_id]['genre'])
    id_genre_mapping[each_id] = genre_list
    
id_studio_mapping = {}
for each_id in all_ids:
    id_studio_mapping[each_id] = list(filtered_df[truncated_df['animeID'] == each_id]['studio'])
    
id_source_mapping = {}
for each_id in all_ids:
    id_source_mapping[each_id] = list(filtered_df[truncated_df['animeID'] == each_id]['source'])
    
id_producers_mapping = {}
for each_id in all_ids:
    id_producers_mapping[each_id] = list(filtered_df[truncated_df['animeID'] == each_id]['producers'])
    
id_rating_mapping = {}
for each_id in all_ids:
    id_rating_mapping[each_id] = list(filtered_df[truncated_df['animeID'] == each_id]['rating'])

1867 unique anime


c:\users\sidac\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
c:\users\sidac\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
c:\users\sidac\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
c:\users\sidac\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
c:\users\sidac\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [6]:
# get distinct df, remove duplicates
reduced_df = filtered_df.groupby('animeID').head(1)

In [7]:
# will add 40 columns to the data
all_genres = set([item for sublist in id_genre_mapping.values() for item in sublist])
len(all_genres) 

40

In [8]:
anime_IDs = reduced_df.animeID.tolist()
genres_new = []
studios_new = []
sources_new = []
producers_new = []
ratings_new = []
for each_id in anime_IDs:
    genres_new.append(id_genre_mapping[each_id])
    studios_new.append(id_studio_mapping[each_id])
    sources_new.append(id_source_mapping[each_id])
    producers_new.append(id_producers_mapping[each_id])
    ratings_new.append(id_rating_mapping[each_id])
reduced_df.head()

,animeID,title_english,type,source,producers,genre,studio,episodes,premiered,rating,score,scored_by,rank,popularity,members,favorites,synopsis
0,1,Cowboy Bebop,TV,Original,Bandai Visual,Action,Sunrise,26.0,Spring 1998,R - 17+ (violence & profanity),8.81,405664,26,39,795733,43460,"In the year 2071, humanity has colonized sever..."
16,6,Trigun,TV,Manga,Victor Entertainment,Action,Madhouse,26.0,Spring 1998,PG-13 - Teens 13 or older,8.30,212537,255,146,408548,10432,"Vash the Stampede is the man with a $$60,000,0..."
22,7,Witch Hunter Robin,TV,Original,Bandai Visual,Action,Sunrise,26.0,Summer 2002,PG-13 - Teens 13 or older,7.33,32837,2371,1171,79397,537,Witches are individuals with special powers li...
28,8,Beet the Vandel Buster,TV,Manga,TV Tokyo,Adventure,Toei Animation,52.0,Fall 2004,PG - Children,7.03,4894,3544,3704,11708,14,It is the dark century and the people are suff...
36,16,Honey and Clover,TV,Manga,Genco,Comedy,J.C.Staff,24.0,Spring 2005,PG-13 - Teens 13 or older,8.12,57065,419,536,172274,3752,"Yuuta, Takumi, and Shinobu share a six-tatami ..."


In [9]:
mlb = MultiLabelBinarizer()
encoded_genres = mlb.fit_transform(genres_new)
encoded_studios = mlb.fit_transform(studios_new)
encoded_sources = mlb.fit_transform(sources_new)
encoded_producers = mlb.fit_transform(producers_new)
encoded_ratings = mlb.fit_transform(ratings_new)

In [10]:
genre_columns_added = encoded_genres.shape[1]

for col_idx in range(genre_columns_added):
    reduced_df.insert(len(reduced_df.columns), "genre_{}".format(col_idx+1), encoded_genres[:, col_idx])

for col_idx in range(encoded_studios.shape[1]):
    reduced_df.insert(len(reduced_df.columns), "studio_{}".format(col_idx+1), encoded_studios[:, col_idx])
    
for col_idx in range(encoded_sources.shape[1]):
    reduced_df.insert(len(reduced_df.columns), "source_{}".format(col_idx+1), encoded_sources[:, col_idx])
    
for col_idx in range(encoded_producers.shape[1]):
    reduced_df.insert(len(reduced_df.columns), "producers_{}".format(col_idx+1), encoded_producers[:, col_idx])
    
for col_idx in range(encoded_ratings.shape[1]):
    reduced_df.insert(len(reduced_df.columns), "rating_{}".format(col_idx+1), encoded_ratings[:, col_idx])

In [14]:
reduced_df.head()

(1867, 878)

# Encoding textual data - tokenization approach
* May want to look into cleaning all the synopsis first, ex: remove (), lower case, etc (disregarding this for now)

In [ ]:
synopsis_list = reduced_df['synopsis'].tolist()
# synopsis_list

In [ ]:
VOCAB_SIZE = None
MAX_SEQ_LEN = 0

# find vocab_size
all_words = {}
for synopsis in synopsis_list:
    word_list = synopsis.split(" ")
    # find max seq len
    if len(word_list) > MAX_SEQ_LEN:
        MAX_SEQ_LEN = len(word_list)
        sent = word_list

    for ea_word in word_list:
        if ea_word in all_words:
            all_words[ea_word] += 1
        else:
            all_words[ea_word] = 1
VOCAB_SIZE = len(all_words.keys())
print ('vocab_size = ', VOCAB_SIZE)
print ('max_seq_len = ', MAX_SEQ_LEN)
# print (MAX_SEQ_LEN, sent)

In [ ]:
encoded_synopsis = [one_hot(x, VOCAB_SIZE) for x in synopsis_list]
padded_synopsis = pad_sequences(encoded_synopsis, maxlen=MAX_SEQ_LEN, padding='pre')
padded_synopsis.shape

In [ ]:
# load pretrained google word2vec model
model = gensim.models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
# get averaged word embedding
EMBED_DIM = 300

all_synopsis_vectors = np.empty((0, EMBED_DIM))
print (all_synopsis_vectors.shape)
for each_synopsis in synopsis_list:
    synopsis_words = each_synopsis.split(" ")
    word_count = len(synopsis_words)
    synopsis_sum_vector = np.zeros((EMBED_DIM,))
    for each_word in synopsis_words:
        try:
            synopsis_sum_vector += model[each_word]
        except:
            # word not in pretrained vocab
            pass
    synopsis_avg_vector = (synopsis_sum_vector / word_count).reshape(1, -1)
    all_synopsis_vectors = np.append(all_synopsis_vectors, synopsis_avg_vector, axis=0)
all_synopsis_vectors.shape

In [ ]:
synopsis_columns_added = all_synopsis_vectors.shape[1]
for col_idx in range(synopsis_columns_added):
    reduced_df.insert(len(reduced_df.columns), "synopsis_embedded_{}".format(col_idx+1), all_synopsis_vectors[:, col_idx])
reduced_df.head()